# exp020_tuner

In [20]:
import os
import sys
import gc
import random
import pathlib
import subprocess
from dataclasses import dataclass
from dotenv import load_dotenv
load_dotenv
sys.path.append(os.getenv('UTILS_PATH'))
from tqdm import tqdm
import multiprocessing
import inspect

import pandas as pd
import numpy as np
import itertools
import cudf
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import line_notify
import optuna.integration.lightgbm as optuna_lgb

In [21]:
import builtins
import types

def imports():
    for name, val in globals().items():
        # module imports
        if isinstance(val, types.ModuleType):
            yield name, val

            # functions / callables
        if hasattr(val, '__call__'):
            yield name, val


def noglobal(f):
    '''
    ref: https://gist.github.com/raven38/4e4c3c7a179283c441f575d6e375510c
    '''
    return types.FunctionType(f.__code__,
                              dict(imports()),
                              f.__name__,
                              f.__defaults__,
                              f.__closure__
                              )

In [22]:
@dataclass
class Cfg:
    exp_name = "exp020"
    seed = 42
    k = 20
    type2id = {"clicks":0, "carts":1, "orders":2}
    id2type = {0:"clicks", 1:"carts", 2:"orders"}
    train_weeks = ["week3"]
    valid_week = "week4"
    valid_session_n = 1_000_000
    input_dir = os.getenv('INPUT_DIR')
    output_dir = os.getenv('OUTPUT_DIR')
    prep_dir = os.getenv("PREP_DIR")

cfg = Cfg()
# os.makedirs(os.path.join(cfg.output_dir, cfg.exp_name), exist_ok=True)
random.seed(cfg.seed)

In [23]:
@noglobal
def evaluate(clicks_labels, carts_labels, orders_labels, 
             clicks_preds, carts_preds, orders_preds, k=20):

    num_clicks = 0
    num_carts = 0
    num_orders = 0
    hit_clicks = 0
    hit_carts = 0
    hit_orders = 0

    for i in range(len(clicks_labels)):
        clicks_label = clicks_labels[i]
        carts_label = carts_labels[i]
        orders_label = orders_labels[i]
        clicks_pred = clicks_preds[i]
        carts_pred = carts_preds[i]
        orders_pred = orders_preds[i]

        if type(clicks_pred) == list:
            clicks_pred = clicks_pred[:k]
        else:
            clicks_pred = []
        if type(carts_pred) == list:
            carts_pred = carts_pred[:k]
        else:
            carts_pred = []    
        if type(orders_pred) == list:
            orders_pred = orders_pred[:k]
        else:
            orders_pred = []

        if not np.isnan(clicks_label):
            num_clicks += 1
            hit_clicks += int(clicks_label in clicks_pred)

        if type(carts_label) == list:
            num_carts += min(len(carts_label), k)
            hit_carts += len(set(carts_pred) & set(carts_label))
            
        if type(orders_label) == list:
            num_orders += min(len(orders_label), k)
            hit_orders += len(set(orders_pred) & set(orders_label))


    recall_clicks = hit_clicks / num_clicks
    recall_carts = hit_carts / num_carts
    recall_orders = hit_orders / num_orders
    w_recall_clicks = recall_clicks * 0.10
    w_recall_carts = recall_carts * 0.30
    w_recall_orders = recall_orders * 0.6
    score = w_recall_clicks + w_recall_carts + w_recall_orders

    results = {}
    results["num_clicks"] = num_clicks
    results["hit_clicks"] = hit_clicks
    results["num_carts"] = num_carts
    results["hit_carts"] = hit_carts
    results["num_orders"] = num_orders
    results["hit_orders"] = hit_orders
    results["recall_clicks"] = format(recall_clicks, ".3f")
    results["recall_carts"] = format(recall_carts, ".3f")
    results["recall_orders"] = format(recall_orders, ".3f")
    results["w_recall_clicks"] = format(w_recall_clicks, ".3f")
    results["w_recall_carts"] = format(w_recall_carts, ".3f")
    results["w_recall_orders"] = format(w_recall_orders, ".3f")
    results["score"] = format(score, ".3f")

    return results

In [24]:
class Candidate:
    def __init__(self, pdf, labels=None):
        self.df = cudf.from_pandas(pdf)
        self.target_sessions = pdf["session"].unique().tolist()
        self.results = pd.DataFrame(columns=["name", "num_clicks", "hit_clicks", "num_carts", "hit_carts", "num_orders", "hit_orders", 
                                             "recall_clicks", "recall_carts", "recall_orders", "w_recall_clicks", "w_recall_carts", "w_recall_orders", "score"])
        self.output = pd.DataFrame(columns=["session", "aid"], dtype=int)
        self.labels = labels

    def _entry(self, new_candidate_df, name, k):
        new_candidate_df[f"{name}_rank"] = new_candidate_df.groupby("session")["session"].cumcount()
        new_candidate_df = new_candidate_df.to_pandas()
        self.output = pd.concat([self.output, new_candidate_df[["session", "aid"]]])
        self.output = self.output.drop_duplicates(subset=["session", "aid"])
        
        self.output = self.output.merge(new_candidate_df, on=["session", "aid"], how="left")

        if self.labels is not None:
            self._eval(new_candidate_df[["session", "aid"]], name, k)

    def _eval(self, new_candidate_df, name, k):
        new_candidate_df = new_candidate_df.groupby("session")["aid"].apply(list).reset_index()
        eval_df = pd.DataFrame(self.target_sessions, columns=["session"])
        eval_df = eval_df.merge(new_candidate_df, on=["session"], how="left")
        assert eval_df["session"].tolist() == self.labels["session"].tolist()
        eval_result = evaluate(self.labels["clicks_labels"].tolist(), self.labels["carts_labels"].tolist(), self.labels["orders_labels"].tolist(),
                               eval_df["aid"].tolist(), eval_df["aid"].tolist(), eval_df["aid"].tolist(), k)
        print(name)
        print(eval_result)
        self.results = pd.concat([self.results, pd.DataFrame([[name] + list(eval_result.values())], columns=["name"] + list(eval_result.keys()))])

    def add(self, strategy, name, usetypes=['clicks', 'carts', 'orders'], trans_map=None, k=20):
        candidate_df = self.df[self.df["type"].isin(usetypes)].copy()
        
        if strategy == "session_frequent":
            candidate_df = self._session_frequent(candidate_df, k)
        elif strategy == "session_latest":
            candidate_df = self._session_latest(candidate_df, k)
        elif strategy == "total_frequent":
            candidate_df = self._total_frequent(candidate_df, k)
        
        if trans_map is not None:
            candidate_df["aid"] = candidate_df["aid"].map(trans_map)
            candidate_df = candidate_df.dropna(subset=["aid"])
            
        self._entry(candidate_df, name, k)

    def _session_frequent(self, candidate_df, k):
        candidate_df = candidate_df.groupby(["session", "aid"])["ts"].count().reset_index()
        candidate_df.columns = ["session", "aid", "aid_count"]
        candidate_df = candidate_df.sort_values(["session", "aid_count", "aid"], ascending=(True, False, True))
        candidate_df = candidate_df[candidate_df.groupby("session")["aid_count"].cumcount() < k].copy()
        candidate_df = candidate_df[["session", "aid"]].copy()
        return candidate_df

    def _session_latest(self, candidate_df, k):
        candidate_df = candidate_df.sort_values(["session", "ts"], ascending=(True, False)).drop_duplicates(subset=["session", "aid"])
        candidate_df = candidate_df.sort_values(["session", "ts"], ascending=(True, False))[candidate_df.groupby("session")["ts"].cumcount() < k].copy()
        candidate_df = candidate_df[["session", "aid"]].copy()
        return candidate_df

    def _total_frequent(self, candidate_df, k):
        candidate_df = candidate_df.groupby("aid")["ts"].count().reset_index()
        candidate_df.columns = ["aid", "aid_count"]
        topk_freq_aids = candidate_df.sort_values("aid_count", ascending=False)["aid"].to_arrow().to_pylist()[:k]
        sessions = []
        aids = []
        for session, aid in itertools.product(self.target_sessions, topk_freq_aids):
            sessions.append(session)
            aids.append(aid)
        candidate_df = cudf.DataFrame({"session": sessions, "aid": aids})
        return candidate_df


In [25]:
@noglobal
def add_labels(df, labels, type_):
    type_labels = labels[["session", f"{type_}_labels"]].dropna().copy()
    type_labels.columns = ["session", "aid"]
    type_labels = type_labels.explode("aid")
    type_labels["labels"] = 1
    df = df.merge(type_labels, on=["session", "aid"], how="left")
    df["labels"] = df["labels"].fillna(0)
    return df    

In [26]:
@noglobal
def add_ground_truth_candidate(df, labels, type_):
    gt = labels[["session", f"{type_}_labels"]].dropna().copy()
    gt.columns = ["session", "aid"]
    gt = gt.explode("aid")
    df = pd.concat([df, gt])
    df = df.drop_duplicates()
    return df

In [27]:
@noglobal
def under_sampling(df, label_col):
    cfg = Cfg()
    pos_df = df[df[label_col]==1].copy()
    neg_df = df[df[label_col]==0].copy()
    pos_n = len(pos_df)
    return pd.concat([pos_df, neg_df.sample(pos_n, random_state=cfg.seed)])

In [28]:
class AddFeatures:
    def __init__(self, sessions, candidate):
        cfg = Cfg()
        self.sessions = cudf.from_pandas(sessions)
        self.output = candidate
        self.type_dict = cfg.type2id

    def add_features(self, features_name):
        if features_name == "session_cnt":
            self._session_cnt()
        elif features_name == "session_aid_nunique":
            self._session_aid_nunique()
        elif features_name == "session_last_type":
            self._session_last_type()
        elif features_name == "aid_cnt":
            self._aid_cnt()
        
    def _session_cnt(self):
        agg_df = self.sessions.groupby(["session", "type"])["ts"].count().reset_index().rename(columns={"ts": "cnt"})
        # session_total_cnt
        features = agg_df.groupby("session")["cnt"].sum().reset_index().rename(columns={"cnt": "session_total_cnt"})
        self.output = self.output.merge(features.to_pandas(), on=["session"], how="left")
        self.output["session_total_cnt"] = self.output["session_total_cnt"].fillna(0)
        # session_{type}_cnt
        for type_ in ["clicks", "carts", "orders"]:
            col_name = f"session_{type_}_cnt"
            features = agg_df[agg_df["type"]==type_].copy()
            features = features.rename(columns={"cnt": col_name})
            features = features[["session", col_name]].copy()
            self.output = self.output.merge(features.to_pandas(), on=["session"], how="left")
            self.output[col_name] = self.output[col_name].fillna(0)

    def _session_aid_nunique(self):
        features = self.sessions.groupby("session")["aid"].nunique().reset_index().rename(columns={"aid": "session_aid_nunique"})
        self.output = self.output.merge(features.to_pandas(), on=["session"], how="left")
        self.output["session_aid_nunique"] = self.output["session_aid_nunique"].fillna(0)
        
    def _session_last_type(self):
        features = self.sessions.groupby("session").nth(-1).reset_index()[["session", "type"]]
        features = features.rename(columns={"type": "session_last_type"})
        features["session_last_type"] = features["session_last_type"].map(self.type_dict)
        self.output = self.output.merge(features.to_pandas(), on=["session"], how="left")

    def _aid_cnt(self):
        agg_df = self.sessions.groupby(["session", "aid", "type"])["ts"].count().reset_index().rename(columns={"ts": "cnt"})

        # aid_total_cnt
        features = agg_df.groupby(["session", "aid"])["cnt"].sum().reset_index().rename(columns={"cnt": "aid_total_cnt"})
        self.output = self.output.merge(features.to_pandas(), on=["session", "aid"], how="left")
        self.output["aid_total_cnt"] = self.output["aid_total_cnt"].fillna(0)

        # aid_{type}_cnt
        for type_ in ["clicks", "carts", "orders"]:
            col_name = f"aid_{type_}_cnt"
            features = agg_df[agg_df["type"]==type_].copy()
            features = features.rename(columns={"cnt": col_name})
            features = features[["session", "aid", col_name]].copy()
            self.output = self.output.merge(features.to_pandas(), on=["session", "aid"], how="left")
            self.output[col_name] = self.output[col_name].fillna(0)     

In [29]:
@noglobal
def make_cand_df(sessions, labels=None, trans_maps=None):
    cfg = Cfg()

    if labels is not None:
         cand = Candidate(sessions)
    else:
        cand = Candidate(sessions, labels)

    cand.add(strategy="session_frequent", name="session_frequent")
    cand.add(strategy="session_latest", name="session_latest")
    cand.add(strategy="total_frequent", name="total_frequent")
    cand.add(strategy="session_frequent", trans_map=trans_maps["pair0"], name="session_frequent_pair")
    
    for i in range(5):
        cand.add(strategy="session_latest", trans_map=trans_maps[f"pair{str(i)}"], name=f"session_latest_pair{str(i)}")
      
    cand.add(strategy="session_frequent", trans_map=trans_maps["clicks_pair"], name="session_frequent_clicks_pair")
    cand.add(strategy="session_latest", trans_map=trans_maps["clicks_pair"], name="session_latest_clicks_pair")
    cand.add(strategy="session_latest", trans_map=trans_maps["carts_pair"], name="session_latest_carts_pair")
    return cand.output

@noglobal
def add_features(sessions, cand_df):
    feat = AddFeatures(sessions, cand_df)
    feat.add_features("session_cnt")
    feat.add_features("session_aid_nunique")
    feat.add_features("session_last_type")
    feat.add_features("aid_cnt")
    return feat.output.copy()

In [30]:
trans_maps = {}
pair_df_org = pd.read_pickle(cfg.prep_dir + "co_visitation_matrix.pkl")
pair_df_org = pair_df_org[pair_df_org["cnt"] > 3].copy()
pair_df_org = pair_df_org.sort_values(["aid_x", "cnt"], ascending=(True, False))

for i in range(5):
    pair_df = pair_df_org.groupby("aid_x").nth(i).reset_index()[["aid_x", "aid_y"]]
    pair_dict = {k: v for k, v in zip(pair_df["aid_x"].tolist(), pair_df["aid_y"].tolist())}
    trans_maps[f"pair{str(i)}"] = pair_dict

pair_df = pd.read_pickle(cfg.prep_dir + "co_visitation_matrix_clicks.pkl")
pair_df = pair_df[pair_df["cnt"] > 3].copy()
pair_df = pair_df.sort_values(["aid_x", "cnt"], ascending=(True, False))
pair_df = pair_df.groupby("aid_x").head(1)[["aid_x", "aid_y"]]
pair_dict = {k: v for k, v in zip(pair_df["aid_x"].tolist(), pair_df["aid_y"].tolist())}
trans_maps["clicks_pair"] = pair_dict

pair_df = pd.read_pickle(cfg.prep_dir + "co_visitation_matrix_carts.pkl")
pair_df = pair_df[pair_df["cnt"] > 3].copy()
pair_df = pair_df.sort_values(["aid_x", "cnt"], ascending=(True, False))
pair_df = pair_df.groupby("aid_x").head(1)[["aid_x", "aid_y"]]
pair_dict = {k: v for k, v in zip(pair_df["aid_x"].tolist(), pair_df["aid_y"].tolist())}
trans_maps["carts_pair"] = pair_dict

pair_df = pd.read_pickle(cfg.prep_dir + "co_visitation_matrix_orders.pkl")
pair_df = pair_df[pair_df["cnt"] > 3].copy()
pair_df = pair_df.sort_values(["aid_x", "cnt"], ascending=(True, False))
pair_df = pair_df.groupby("aid_x").head(1)[["aid_x", "aid_y"]]
pair_dict = {k: v for k, v in zip(pair_df["aid_x"].tolist(), pair_df["aid_y"].tolist())}
trans_maps["orders_pair"] = pair_dict

# trainデータ作成

In [31]:
report = f"\n{cfg.exp_name}\n"
report += f'train_start\n'
line_notify.send(report)

In [32]:
train_weeks = cfg.train_weeks
train_session_ids = []
dfs = []
for weeks in train_weeks:
    # データ読み込み
    week_sessions = pd.read_pickle(os.getenv("PREP_DIR") + f"train_sessions_{weeks}.pkl")
    week_labels = pd.read_pickle(os.getenv("PREP_DIR") + f"labels_{weeks}.pkl")
    train_session_ids.extend(week_sessions["session"].unique().tolist())

    for type_, type_int in zip(["clicks", "carts", "orders"], [0, 1, 2]):
        # 候補選出
        cand_df = make_cand_df(week_sessions, trans_maps=trans_maps)

        # 正解追加・under sampling
        # cand_df = add_ground_truth_candidate(cand_df, week_labels, type_)
        cand_df = add_labels(cand_df, week_labels, type_)
        cand_df = under_sampling(cand_df, "labels")

        # 特徴量付与
        train_tmp = add_features(week_sessions, cand_df)
        train_tmp["type"] = type_int
        del cand_df
        gc.collect()
        dfs.append(train_tmp)
train = pd.concat(dfs)
train = train.reset_index(drop=True)
train_session_ids = list(set(train_session_ids))

del dfs, week_sessions, week_labels
gc.collect()

0

# validデータ作成

In [33]:
valid_week = cfg.valid_week
dfs = []

# データ読み込み
week_sessions = pd.read_pickle(os.getenv("PREP_DIR") + f"train_sessions_{valid_week}.pkl")
week_labels = pd.read_pickle(os.getenv("PREP_DIR") + f"labels_{valid_week}.pkl")
valid_session_ids = week_sessions["session"].unique().tolist()

# validに使うsessionを絞る (trainに存在するsessionを除外した上で設定した数に絞る)
valid_session_n = min(cfg.valid_session_n, len(valid_session_ids))
valid_session_ids = random.sample(valid_session_ids, valid_session_n)
week_sessions = week_sessions[week_sessions["session"].isin(valid_session_ids)].copy()
week_labels = week_labels[week_labels["session"].isin(valid_session_ids)].copy()

for type_, type_int in zip(["clicks", "carts", "orders"], [0, 1, 2]):
    # 候補選出
    cand_df = make_cand_df(week_sessions, trans_maps=trans_maps)

    # ラベル付与
    cand_df = add_labels(cand_df, week_labels, type_)
    
    # 特徴量付与
    train_tmp = add_features(week_sessions, cand_df)
    train_tmp["type"] = type_int
    del cand_df
    gc.collect()

    dfs.append(train_tmp)
valid = pd.concat(dfs)
valid = valid.reset_index(drop=True)

del dfs, week_sessions
gc.collect()

0

# 学習

In [34]:
target = "labels"
not_use_cols = ["session", "aid", target]
features = [c for c in train.columns if c not in not_use_cols]

In [35]:
params = {
   'objective': 'binary',
   'boosting': 'gbdt',
   'learning_rate': 0.1,
   'metric': 'binary_logloss',
   'seed': cfg.seed
}

In [36]:
# 学習
vl_pred_df = valid[["session", "aid", "type"]].copy()

tr_x, tr_y = train[features], train[target]
vl_x, vl_y = valid[features], valid[target]
tr_data = lgb.Dataset(tr_x, label=tr_y)
vl_data = lgb.Dataset(vl_x, label=vl_y)

model = optuna_lgb.train(params, tr_data, valid_sets=[tr_data, vl_data],
                         num_boost_round=2000000, early_stopping_rounds=100, verbose_eval=1000)

[I 2022-11-24 10:13:49,213] A new study created in memory with name: no-name-b33e4239-dff7-4815-b000-e3e6a1512955
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 3312365, number of negative: 3312365
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.267990 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1010
[LightGBM] [Info] Number of data points in the train set: 6624730, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training until validation scores don't improve for 100 rounds
[1000]	valid_0's binary_logloss: 0.294831	valid_1's binary_logloss: 0.297561
[2000]	valid_0's binary_logloss: 0.292422	valid_1's binary_logloss: 0.29706


feature_fraction, val_score: 0.296994:  14%|#4        | 1/7 [13:51<1:23:09, 831.54s/it]

Early stopping, best iteration is:
[2291]	valid_0's binary_logloss: 0.291834	valid_1's binary_logloss: 0.296994
[LightGBM] [Info] Number of positive: 3312365, number of negative: 3312365
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.334738 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1010
[LightGBM] [Info] Number of data points in the train set: 6624730, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training until validation scores don't improve for 100 rounds
[1000]	valid_0's binary_logloss: 0.294588	valid_1's binary_logloss: 0.297524
[2000]	valid_0's binary_logloss: 0.292084	valid_1's binary_logloss: 0.296989


feature_fraction, val_score: 0.296984:  29%|##8       | 2/7 [25:11<1:01:51, 742.26s/it]

...

Early stopping, best iteration is:
[1466]	valid_0's binary_logloss: 0.29173	valid_1's binary_logloss: 0.29636
[LightGBM] [Info] Number of positive: 3312365, number of negative: 3312365
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.357666 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1010
[LightGBM] [Info] Number of data points in the train set: 6624730, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training until validation scores don't improve for 100 rounds
[1000]	valid_0's binary_logloss: 0.293214	valid_1's binary_logloss: 0.296699


min_data_in_leaf, val_score: 0.296332:  80%|########  | 4/5 [36:57<09:07, 547.83s/it]

Early stopping, best iteration is:
[1012]	valid_0's binary_logloss: 0.293174	valid_1's binary_logloss: 0.29664
[LightGBM] [Info] Number of positive: 3312365, number of negative: 3312365
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.383575 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1010
[LightGBM] [Info] Number of data points in the train set: 6624730, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training until validation scores don't improve for 100 rounds
[1000]	valid_0's binary_logloss: 0.29322	valid_1's binary_logloss: 0.296654


min_data_in_leaf, val_score: 0.296332: 100%|##########| 5/5 [47:31<00:00, 570.25s/it]

Early stopping, best iteration is:
[1466]	valid_0's binary_logloss: 0.291759	valid_1's binary_logloss: 0.296347


In [37]:
print(model.params)

{'objective': 'binary', 'boosting': 'gbdt', 'learning_rate': 0.1, 'metric': 'binary_logloss', 'seed': 42, 'feature_pre_filter': False, 'lambda_l1': 0.0035714307987461313, 'lambda_l2': 8.837218918956209, 'num_leaves': 56, 'feature_fraction': 0.8839999999999999, 'bagging_fraction': 0.8695840030320283, 'bagging_freq': 2, 'min_child_samples': 20, 'num_iterations': 2000000, 'early_stopping_round': 100}
